In [1]:
from pathlib import Path
from functools import partial

from utils import load_polar

import pybem as pb
import numpy as np
import pandas as pd

In [2]:
PATH_POLARS = Path("../polars")
file_lift = "lift.csv"
file_drag = "drag.csv"

In [3]:
csv_reader = partial(pd.read_csv, sep=";", index_col=None)
load_polar = partial(load_polar, path = PATH_POLARS, file_lift = file_lift, file_drag = file_drag, reader = csv_reader)

In [4]:
# Define known sections
sections = [
    pb.Section(
        name="Hub",
        r=0.3,
        beta=60,
        chord=0.4,
        airfoil=pb.Airfoil(
            polar_cl=load_polar(airfoil_type="765", which="cl"),
            polar_cd=load_polar(airfoil_type="765", which="cd"),
        ),
    ),
    pb.Section(
        name="Station 0",
        r=0.3,
        beta=60,
        chord=0.4,
        airfoil=pb.Airfoil(
            polar_cl=load_polar(airfoil_type="765", which="cl"),
            polar_cd=load_polar(airfoil_type="765", which="cd"),
        ),
    ),
    pb.Section(
        name="Station 1",
        r=0.6,
        beta=45,
        chord=0.35,
        airfoil=pb.Airfoil(
            polar_cl=load_polar(airfoil_type="764", which="cl"),
            polar_cd=load_polar(airfoil_type="764", which="cd"),
        ),
    ),
    pb.Section(
        name="Station 2",
        r=0.6,
        beta=45,
        chord=0.35,
        airfoil=pb.Airfoil(
            polar_cl=load_polar(airfoil_type="763", which="cl"),
            polar_cd=load_polar(airfoil_type="763", which="cd"),
        ),
    ),
    pb.Section(
        name="Tip",
        r=1.2,
        beta=30,
        chord=0.2,
        airfoil=pb.Airfoil(
            polar_cl=load_polar(airfoil_type="762", which="cl"),
            polar_cd=load_polar(airfoil_type="762", which="cd"),
        ),
    ),
]

In [10]:
# Define propeller
B = 6
propeller = pb.Propeller(B=B, sections=sections)

# Define flow conditions and BEM method
J = 0.4
bem = pb.BladeElementMethod(J=J, propeller=propeller, tip_loss=False, hub_loss=False)

# Solve
bem.solve()

# Compute forces
CT, CQ = bem.integrate_forces()

ValueError: A value in x_new is below the interpolation range.

In [9]:
CT

4.577199588595352